# 환경설정

In [ ]:
%cd /content/drive/MyDrive/datasets/

/content/drive/MyDrive/datasets


In [ ]:
!ls

dog1.jpg  food10  food10.zip  single_prediction  test_set  training_set  vgg16.h5


## unzip

In [ ]:
!unzip food10.zip -d ./food10

Archive:  food10.zip
replace ./food10/bibimbap/train_086728.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 라이브러리 불러오기(데이터 처리)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

# Callback 처리
from keras.callbacks import ModelCheckpoint

In [2]:
# 학습 데이터(Train) 경로
data_dir = '/content/drive/MyDrive/datasets/food10/'
data = tf.keras.preprocessing.image_dataset_from_directory(data_dir)

Found 5113 files belonging to 10 classes.


In [3]:
data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [4]:
# 데이터 인공지능이 학습 가능한 형태로 변환
datagen = ImageDataGenerator( 
    rescale            =1./255,
    rotation_range     =40,
    width_shift_range  =0.2,
    height_shift_range =0.2,
    shear_range        =0.2,
    zoom_range         =0.2,
    horizontal_flip    =True,
    validation_split   =0.2
)

In [5]:
width       = 228
height      = 228
channels    = 3
batch_size  = 32
image_shape = (width, height, channels)
image_size  = (width, height)


In [6]:
# 데이터 변환
train_data = datagen.flow_from_directory(
    data_dir,
    target_size = image_size,
    batch_size  = batch_size,
    class_mode  = 'categorical',
    subset      = 'training', # 나눌 때 Train으로 사용
    shuffle     = True
)


Found 4094 images belonging to 10 classes.


In [7]:
train_data.class_indices

{'bibimbap': 0,
 'chiffon_cake': 1,
 'donut': 2,
 'egg_roll': 3,
 'garlic_bread': 4,
 'gyoza': 5,
 'kabob': 6,
 'omelette': 7,
 'pizza': 8,
 'sashimi': 9}

In [8]:
val_data = datagen.flow_from_directory(
    data_dir,
    target_size = image_size,
    batch_size  = batch_size,
    class_mode  = 'categorical',
    subset      = 'validation', # 나눌 때 validation으로 사용
    shuffle     = True
)

Found 1019 images belonging to 10 classes.


In [9]:
val_data.class_indices # 5113 -> 4094(train) / 1019(valid)

{'bibimbap': 0,
 'chiffon_cake': 1,
 'donut': 2,
 'egg_roll': 3,
 'garlic_bread': 4,
 'gyoza': 5,
 'kabob': 6,
 'omelette': 7,
 'pizza': 8,
 'sashimi': 9}

In [10]:
# 클래스 명칭 설정
num_classes = len(data.class_names)

In [11]:
num_classes

10

In [12]:
data.class_names

['bibimbap',
 'chiffon_cake',
 'donut',
 'egg_roll',
 'garlic_bread',
 'gyoza',
 'kabob',
 'omelette',
 'pizza',
 'sashimi']

In [ ]:
# 학습할 이미지에 대한 Preview
def show_image(data):
  plt.figure(figsize=(15, 15))
  for images, labels in data.take(1):
    for i in range(9):
      ax = plt.subplot(3, 3, i + 1)
      plt.imshow(images[i].numpy().astype("uint8"))
      ax.axis("off")

show_image(data)

Output hidden; open in https://colab.research.google.com to view.

# 인공지능 알고리즘 설계

In [13]:
pre_model = InceptionV3(
    weights     = 'imagenet',
    include_top = False,
    input_shape = image_shape,
    pooling     = 'avg'
)

In [14]:
pre_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 228, 228, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 113, 113, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 113, 113, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 113, 113, 32)         0         ['batch_normalizati

In [15]:
# 모델 설계
x = pre_model.output
x = BatchNormalization(
    axis     = -1,
    momentum = 0.99,
    epsilon  = 0.001)(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

In [16]:
# 모델 컴파일
model = Model(inputs = pre_model.input, outputs = predictions)

In [17]:
model.compile(
    optimizer = Adam(learning_rate = 0.001), # if learning_rate = 0.0001 -> ?
    loss      = 'categorical_crossentropy',
    metrics   = ['accuracy']
)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 228, 228, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 113, 113, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 113, 113, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 113, 113, 32)         0         ['batch_normalization[0][0

In [19]:
# 단위한 학습량을 변수로 조절
STEP_SIZE_TRAIN = train_data.n // train_data.batch_size # ex) 0/256, 1/256, ... <<< 이 값을 조절
STEP_SIZE_VALID = val_data.n   // val_data.batch_size

In [20]:
STEP_SIZE_TRAIN, STEP_SIZE_VALID

(127, 31)

In [21]:
EPOCHS = 10
filename = '/content/drive/MyDrive/train/ipv3_epoch_{epoch}.h5'
checkpoint = ModelCheckpoint(
    filename,
    monitor         = 'val_loss',
    verbose         = 1,
    save_best_only  = True,
    mode            = 'auto'
)

In [ ]:
history = model.fit(
    train_data,
    steps_per_epoch   = STEP_SIZE_TRAIN,
    validation_data   = val_data,
    validation_steps  = STEP_SIZE_VALID,
    epochs            = EPOCHS,
    verbose           = 1,
    callbacks         = [checkpoint]
)

Epoch 1/10
127/127 [==============================] - ETA: 0s - loss: 3.3114 - accuracy: 0.3055 
Epoch 1: val_loss improved from inf to 228.88268, saving model to /content/drive/MyDrive/train/ipv3_epoch_1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


127/127 [==============================] - 2355s 18s/step - loss: 3.3114 - accuracy: 0.3055 - val_loss: 228.8827 - val_accuracy: 0.1573
Epoch 2/10
 50/127 [==========>...................] - ETA: 22:01 - loss: 3.4618 - accuracy: 0.2631